## Librerias

In [ ]:
import polars as pl
import os
import gcsfs
import os

## Polars

In [ ]:
path_data = os.path.join(os.getcwd(), 'data') 

df = pl.read_csv(path_data + '/marketing_data.csv')
df = df.with_columns(pl.col("Dt_Customer").str.strptime(pl.Date, "%m/%d/%y"))
df = df.rename({col: col.replace(' ', '').replace('_', '') for col in df.columns})
df = df.with_columns(
    pl.col("Income")
    .str.replace_all(r'[$,]', '')
    .str.replace_all(' ', '')
    .cast(pl.Float64)
    .fill_null(0)
)
df.head()

## Google Cloud Storage

In [ ]:
credentials = 'secretos/credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials

fs = gcsfs.GCSFileSystem()
destination = "gs://data-hist-cf/dlt/deltamkt.parquet"
with fs.open(destination, 'wb') as f:
    df.write_parquet(f)